<a href="https://colab.research.google.com/github/1pawn0/YTMusic-Concurrent-Playlist-Downloader-via-yt-dlp/blob/main/YTMusic_Concurrent_Playlist_Downloader_via_yt_dlp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!sudo add-apt-repository ppa:ubuntuhandbook1/ffmpeg8 -y
!sudo apt update -y
!sudo apt install ffmpeg -y
!pip install -qU "yt-dlp[default]"

In [ ]:
# @title Youtube Music Playlist Downloader {"vertical-output":true,"form-width":"50%"}
import yt_dlp
import shutil
import concurrent.futures
from pathlib import Path

# Enter the playlist URL here:
PLAYLIST_URL: str = "https://music.youtube.com/playlist?list=PLAYLIST-ID"  # @param PLAYLIST_URL {"type":"string","placeholder":"https://music.youtube.com/playlist?list=PLAYLIST-ID"}

# Enter the directory path to save the downloaded playlist:
DOWNLOAD_PATH: str = "/path/to/save/downloaded/playlist"  # @param DOWNLOAD_PATH {"type":"string","placeholder":"/path/to/save/downloaded/playlist"}

# Number of playlist items to download in each parallel batch.
BATCH_SIZE: int = 16 # @param BATCH_SIZE {"type":"integer", "placeholder":"8"}

def get_playlist_info(playlist_url: str = PLAYLIST_URL) -> dict:
    ydl_opts = {
        "extract_flat": True,  # do NOT process url_results further. Here `extract_flat` MUST be `True` for efficiency.
        "quiet": True,
        "simulate": True,
        "skip_download": True,
        "restrictfilenames": True,
        "fixup": "detect_or_warn",
    }
    with yt_dlp.YoutubeDL(ydl_opts) as ydl:
        info: dict = ydl.extract_info(playlist_url, download=False)
        return info

def download_playlist_items_in_range(
    playlist_items_range: tuple[int, int],
    playlist_url: str = PLAYLIST_URL,
    dl_path: Path = Path(DOWNLOAD_PATH),
    output_template: str = "%(playlist_title)s/%(title)s.%(ext)s",
):
    ydl_opts: dict = {
        "cookiefile": "/content/cookies.txt", # `cookies.txt` file can be generated by `Get cookies.txt LOCALLY` extension for Chrome
        "playliststart": playlist_items_range[0],
        "playlistend": playlist_items_range[1],
        "outtmpl": output_template,
        "format": "bestaudio/best",
        "format_sort": ["hasaud", "quality", "asr", "abr"],
        "postprocessors": [
            {
                "key": "FFmpegExtractAudio",
                "preferredcodec": "mp3",
                "preferredquality": "0",
            }
        ],
        "paths": {"home": dl_path.__str__()},
        "restrictfilenames": True,
        "nooverwrites": True,
        "fixup": "detect_or_warn",
        # "sleep_interval": 0.1,
        # "max_sleep_interval": 1.1,
        # "extract_flat": False,
        # "verbose": False,
    }

    with yt_dlp.YoutubeDL(ydl_opts) as ydl:
        ydl.download([playlist_url])


def make_zip_archive(dir_path: Path) -> Path:
    if not dir_path.is_dir():
        raise ValueError(f"The provided path '{dir_path}' is not a valid directory.")

    archive_path: Path = Path(shutil.make_archive(base_name=dir_path, format="zip", root_dir=dir_path))
    return archive_path


In [ ]:
pl_info = get_playlist_info()
pl_items_count = pl_info["playlist_count"]
pl_title = pl_info["title"]

download_ranges = []
for i in range(1, pl_items_count + 1, BATCH_SIZE):
    start_index = i
    end_index = min(i + BATCH_SIZE - 1, pl_items_count)
    download_ranges.append((start_index, end_index))

In [ ]:
with concurrent.futures.ThreadPoolExecutor(max_workers=16) as executor:
    executor.map(download_playlist_items_in_range, download_ranges)
